<a href="https://colab.research.google.com/github/MatheusHenriqueDaSilva/data_science/blob/main/house_random_forest_regressor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

prevendo preço de casas usando random forest regressor

In [1]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
from sklearn.model_selection import train_test_split

In [62]:
# Data understanding
DATA = pd.read_csv('/content/train.csv')
DATA.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [63]:
# Data understanding
test_submit = pd.read_csv('/content/test.csv')
test_submit.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [6]:
DATA.shape

(1460, 81)

In [119]:
def generate_metadata(dataframe):

    metadata2 = pd.DataFrame({
        'nome_variavel': dataframe.columns,
        'tipo': dataframe.dtypes,
        'qt_nulos': dataframe.isnull().sum(),
        'percent_nulos': round((dataframe.isnull().sum() / len(dataframe))* 100,2),
        'cardinalidade': dataframe.nunique(),
    })
    metadata2=metadata2.sort_values(by='tipo')
    metadata2 = metadata2.reset_index(drop=True)

    return metadata2

metadados = generate_metadata(DATA)
metadados.tail(50)

,nome_variavel,tipo,qt_nulos,percent_nulos,cardinalidade
31,OverallQual,int64,0,0.00,10
32,OverallCond,int64,0,0.00,9
33,YearBuilt,int64,0,0.00,112
34,YearRemodAdd,int64,0,0.00,61
35,GarageYrBlt,float64,81,5.55,97
36,LotFrontage,float64,259,17.74,110
37,MasVnrArea,float64,8,0.55,327
38,Foundation,object,0,0.00,6
39,GarageFinish,object,81,5.55,3
40,LandSlope,object,0,0.00,3


In [214]:
# Realizando HOLD-OUT da base, buscando conseguir avaliar o modelo posteriormente.
lista_spec = ['Id','SalePrice']
target = DATA['SalePrice']
data_var = DATA.drop(lista_spec,axis=1)

X_train, X_test, Y_train, Y_test = train_test_split(data_var,
                                                    target,
                                                    test_size=0.4,
                                                    random_state=42)
X_train.shape,X_test.shape,Y_train.shape,Y_test.shape


((876, 79), (584, 79), (876,), (584,))

In [215]:
# Variáveis numéricas
# Filtrando o DataFrame
metadados3 = generate_metadata(X_train)
missing_cutoff = 90
df_numericas_drop_nulos = metadados3[(metadados3['percent_nulos'] >= missing_cutoff) | (metadados3['cardinalidade'] >= 4)]
lista_drop_nulos_numericas = list(df_numericas_drop_nulos.nome_variavel.values)
lista_drop_nulos_numericas

['MSSubClass',
 'GrLivArea',
 'Fireplaces',
 'TotRmsAbvGrd',
 'BsmtFinSF1',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 'BedroomAbvGr',
 'FullBath',
 '1stFlrSF',
 '2ndFlrSF',
 'LowQualFinSF',
 'GarageArea',
 'YearRemodAdd',
 'GarageCars',
 'OverallCond',
 'YrSold',
 'LotArea',
 'MoSold',
 'MiscVal',
 'PoolArea',
 'ScreenPorch',
 '3SsnPorch',
 'EnclosedPorch',
 'YearBuilt',
 'OpenPorchSF',
 'BsmtFullBath',
 'WoodDeckSF',
 'OverallQual',
 'GarageYrBlt',
 'MasVnrArea',
 'LotFrontage',
 'FireplaceQu',
 'GarageType',
 'PoolQC',
 'Fence',
 'MiscFeature',
 'GarageQual',
 'GarageCond',
 'Functional',
 'KitchenQual',
 'HeatingQC',
 'MSZoning',
 'Alley',
 'LotShape',
 'LandContour',
 'LotConfig',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'Electrical',
 'RoofStyle',
 'Exterior1st',
 'Exterior2nd',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Heating',
 'SaleType',
 'RoofMatl',
 'SaleCo

In [216]:
# retirando lista de variáveis com alto percentual de nulos
X_train_null = X_train.drop(axis=1,columns=lista_drop_nulos_numericas)

with open('/content/list_null.pkl', 'wb') as f:
    pickle.dump(lista_drop_nulos_numericas, f)
X_train_null.shape

(876, 10)

In [217]:
#Aplivando drop na base de teste
X_test_null = X_test.drop(axis=1,columns=lista_drop_nulos_numericas)
X_test_null.shape

(584, 10)

In [218]:
metadados2 = generate_metadata(X_train_null)
card_cutoff = 0
df_categ_labelenc = metadados2[(metadados2['cardinalidade'] > card_cutoff) & (metadados2['tipo'] == 'object')]
lista_vars_abt = list(X_train.columns)
lista_lenc = list(df_categ_labelenc.nome_variavel.values)


print('Lista de vars para Label Encoding: ',len(lista_lenc))

Lista de vars para Label Encoding:  7


In [219]:
metadados2

,nome_variavel,tipo,qt_nulos,percent_nulos,cardinalidade
0,BsmtHalfBath,int64,0,0.00,3
1,HalfBath,int64,0,0.00,3
2,KitchenAbvGr,int64,0,0.00,3
3,Street,object,0,0.00,2
4,Utilities,object,0,0.00,2
5,LandSlope,object,0,0.00,3
6,MasVnrType,object,509,58.11,3
7,CentralAir,object,0,0.00,2
8,GarageFinish,object,46,5.25,3
9,PavedDrive,object,0,0.00,3


In [220]:
from sklearn.preprocessing import LabelEncoder

encoders = {}

for col in lista_lenc:
    encoder = LabelEncoder()
    X_train_null[col] = encoder.fit_transform(X_train_null[col])

    # Armazena o encoder para a coluna atual em um dicionário
    encoders[col] = encoder

# Salva o dicionário de encoders e a lista de colunas em um arquivo .pkl
data_to_serialize = {
    'encoders': encoders,
    'columns': lista_lenc
}

with open('/content/labelenc.pkl', 'wb') as f:
    pickle.dump(data_to_serialize, f)
data_to_serialize

{'encoders': {'Street': LabelEncoder(),
  'Utilities': LabelEncoder(),
  'LandSlope': LabelEncoder(),
  'MasVnrType': LabelEncoder(),
  'CentralAir': LabelEncoder(),
  'GarageFinish': LabelEncoder(),
  'PavedDrive': LabelEncoder()},
 'columns': ['Street',
  'Utilities',
  'LandSlope',
  'MasVnrType',
  'CentralAir',
  'GarageFinish',
  'PavedDrive']}

In [221]:
# Carregar os encoders e a lista de colunas
with open('/content/labelenc.pkl', 'rb') as f:
    loaded_data = pickle.load(f)

loaded_encoders = loaded_data['encoders']
loaded_columns = loaded_data['columns']

# APLICANDO O LABELENCOLDING NA BASE DE TESTE
for col in loaded_columns:
    if col in loaded_encoders:
        # Transforma a coluna usando o encoder carregado
        X_test_null[col] = loaded_encoders[col].transform(X_test_null[col])



In [224]:
# criando o melhor preditor de light GBM
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

x_treino_final = X_train_null
y_treino_final = Y_train
x_teste_final = X_test_null
y_teste_final = Y_test

# Definindo o modelo
model = RandomForestRegressor(random_state=42)

# Definindo os parâmetros para o grid search
param_grid = {
    'n_estimators': [10, 50],
    'max_depth': [2, 3, 5],
    'min_samples_split': [0.001, 0.01],
    'min_samples_leaf': [2, 0.01],
    'bootstrap': [True, False],
}

# Definindo o objeto GridSearchCV
grid = GridSearchCV(model,
                    param_grid,
                    cv=5, # diminui o numero de folds por conta do baixo número de observações
                    verbose=2,
                    scoring='neg_mean_absolute_error',  #'max_error','neg_mean_absolute_error','neg_mean_squared_error','neg_root_mean_squared_error','neg_mean_squared_log_error'
#,'neg_root_mean_squared_log_error','neg_median_absolute_error','r2','neg_mean_poisson_deviance','neg_mean_gamma_deviance','neg_mean_absolute_percentage_error','d2_absolute_error_score'

                    n_jobs=-1)

# Treinando o modelo com o grid search
grid.fit(x_treino_final, y_treino_final)

# Exibindo os melhores parâmetros encontrados pelo grid search
print("Melhores Parâmetros: ", grid.best_params_)

# Exibindo a melhor pontuação (score) atingida pelo modelo com os melhores parâmetros
print("Melhor erro: ", grid.best_score_)

# Utilizando o melhor modelo para fazer previsões
predictions = grid.best_estimator_.predict(x_teste_final)



best_model = grid.best_estimator_

Fitting 5 folds for each of 48 candidates, totalling 240 fits
Melhores Parâmetros:  {'bootstrap': True, 'max_depth': 5, 'min_samples_leaf': 2, 'min_samples_split': 0.01, 'n_estimators': 50}
Melhor AUC:  -41201.74339043072


In [227]:
with open('/content/best_model.pkl', 'wb') as file:
  pickle.dump(best_model, file)
best_model

RandomForestRegressor(max_depth=5, min_samples_leaf=2, min_samples_split=0.01,
                      n_estimators=50, random_state=42)

In [254]:
from sklearn import metrics
predictions = best_model.predict(x_teste_final)
predictions2 = best_model.predict(x_treino_final)

metrics_df = pd.DataFrame({
    'Conjunto': ['Treino', 'Teste'],
    'média': [predictions.mean(),y_teste_final.mean()],
    'r2': [metrics.r2_score(y_treino_final, predictions2),metrics.r2_score(y_teste_final, predictions,)],
    'MAE': [metrics.mean_absolute_error(y_treino_final, predictions2),metrics.mean_absolute_error(y_teste_final, predictions)],
    'MSE': [metrics.mean_squared_error(y_treino_final, predictions2),metrics.mean_squared_error(y_teste_final, predictions)],
    'RMSE': [np.sqrt(metrics.mean_squared_error(y_treino_final, predictions2)), np.sqrt(metrics.mean_squared_error(y_teste_final, predictions))],
    'MAPE': [metrics.mean_absolute_percentage_error(y_treino_final, predictions2), metrics.mean_absolute_percentage_error(y_teste_final, predictions)]


    })
metrics_df

,Conjunto,média,r2,MAE,MSE,RMSE,MAPE
0,Treino,178532.338721,0.436175,39463.279978,3.200876e+09,56576.283787,0.223604
1,Teste,182420.309932,0.407193,42090.503844,4.296440e+09,65547.233499,0.235363
